In [1]:
using Plots
using SparseArrays
using LinearAlgebra

In [1]:
function source_function(x)
    #=X0 = 60.0
    σ = 0.8
    A = 10.
    return  A .* exp.(.-((x .- X0).^2)./σ^2)=#
    return 0. .* x
end

source_function (generic function with 1 method)

In [3]:
function source(N)
#Parameter
    l = 80.
    dx = l/N
#Mesh
    X_vect = Vector(0:dx:l)
# Function source
    f = source_function(X_vect)
    return f
end

source (generic function with 1 method)

In [3]:
function exact_function(x) #Without Source 
    return 0.5 .* x .+ 10
end

exact_function (generic function with 1 method)

In [4]:
function exact_solution(N)
#Parameter
    l = 80.
    dx = l/N
#Mesh
    X  = Vector(0:dx:l)
#Function 
    f = exact_function(X)
    return f 
end

exact_solution (generic function with 1 method)

In [3]:
function matrix(n) #8 times faster than before (calculate with @btime)

# Parameter for a copper bar 
    d= 8.92 #density 
    sh= 0.092 #specific_heat
    tc= 0.95 #thermal_conductivity
    l= 80 #length
    c = ( tc / (sh * d))^(1/2)
    dx = l/ n  
    
# Build of the tridiagonal Matrix
    A = spdiagm(-1 =>  -(c^2 / dx^2)* ones(n), 0 => 2 * (c^2 / dx^2) * ones(n+1), 1 => - (c^2 / dx^2) * ones(n)) 
    A[1,1] = 1
    A[1,2] = 0
    A[n+1,n+1] = 1 
    A[n+1,n] = 0
    return A
end

matrix (generic function with 1 method)

In [6]:
function heat(N)

#Parameter of the bar 
    l= 80 #length
    dx = l/ N  

# Build of the matrix A
    A = matrix(N)
    #print(A)

# Vector B from Au = B (Boundary)
    b = zeros(N+1)
    b[1] = 10
    b[end] = 50

# Vector S for the source power
    X_vect = Vector(0:dx:l)
    S = source_function(X_vect) 
    S[1] = 0                        #Not sure about this 
    S[end] = 0                      #Not sure about this 
                                    #Dont know if it's B[1] = 10 or S[1] + B[1] = U[1] = 50 same for n
    
# Solving of the Linear System
    u = A \ (b+S)   
    return u 
end 

heat (generic function with 1 method)

In [ ]:
function error_relative(N)
    exact = exact_solution(N)
    modele = heat(N)
    l = 80.
    dx = l / N
    X = Vector(0:dx:l)
    error = (abs.(exact .- modele) ./ exact) * 100
    title = "Percenatge of the relative error for $N points"
    title_2 = "plot_heat_error_$N.png"
    plot(X, error, label = "Percentage of the relative error", xlabel = "Length of the beam", ylabel = "Error", title = title)
    #savefig(title_2)
end

In [2]:
function curves(N)
    l = 80.0
    dx = l/ N 
    exact = exact_solution(N)
    modele = heat(N)
    title = "Heat modelisation for $N points"
    title_2 = "plot_heat_$N.png"
    X = Vector(0:dx:l)
    plot(X, exact, label = "exact_solution", xlabel = "Lenght of the beam", ylabel = "Heat", title = title)
    plot!(X, modele, label = "modele")
    #savefig(title_2)
end 

curves (generic function with 1 method)